In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t
from scipy.stats import norm
from scipy.stats import chi2

Generalized method of moments with serially uncorrelated observations

1. Data simulation

In [2]:
nu = 11
r = t.rvs(nu, size = 1000)

2. Define the moment condition

In [3]:
def moment_cond(nu, data):
    h_1 = data**2-nu/(nu-2)
    h_2 = data**4-(3*nu**2)/((nu-2)*(nu-4))
    Moment_conditions = np.vstack((h_1, h_2))
    Moment_conditions = Moment_conditions.transpose()
    return Moment_conditions

In [4]:
def g_hat(nu, data):
    T = len(data)
    h_1 = (1/T)*sum(data**2)-nu/(nu-2)
    h_2 = (1/T)*sum(data**4)-(3*nu**2)/((nu-2)*(nu-4))
    Moment_conditions = np.vstack((h_1, h_2))
    return Moment_conditions

In [5]:
def obj_func(nu, data, Omega):
    T = len(data)
    h_1 = (1/T)*sum(data**2)-nu/(nu-2)
    h_2 = (1/T)*sum(data**4)-(3*nu**2)/((nu-2)*(nu-4))
    g_bar = np.hstack((h_1, h_2))
    invOmega = np.linalg.inv(Omega)
    Obj_Fun = g_bar @ invOmega @ g_bar.transpose()
    return Obj_Fun

In [6]:
beta = np.array([4.5])
bnds = [(4.00001, None)]
MaxIter = 10000;
GMM_toll1 = 0.000000003
GMM_toll2 = 0.000000004
k = 2                                   # Number of conditions
Omega = np.identity(k)                  # Initial weigthting matrix
T = len(r)
args = (r, Omega)
Init_func = 10000
GMM_lags = 0                            # for serially incorrelated observation choose GMM_lags = 0
i = 1
while i <= MaxIter:
    res = minimize(obj_func, beta, args = args, bounds = bnds, options = {'gtol': 1e-6, 'disp': True})
    if abs(res.fun-Init_func<GMM_toll1) and abs(res.x-beta<GMM_toll2):
        break
    H = moment_cond(res.x,r)
    mH = np.zeros((1,k))
    for j in range(0,k):
        mH[0,j] = np.mean(H[:,j])
    H = H - np.kron(mH, np.ones((H.shape[0],1)))
    Omega = (H.transpose() @ H)/T
    for j in range(0,GMM_lags):
        Gamma = (H[j+1:T, :].transpose() @ H[0:T-j-1, :])/T
        Omega = Omega + (1-(j+1)/(GMM_lags+1))*(Gamma + Gamma.transpose())
    Init_func = res.fun
    beta = res.x
    i = i + 1

In [7]:
# Gradient (forward)
h = 0.00001
pos = np.identity(len(beta))
Grad = np.zeros((k, len(beta)))
for i in range(len(beta)):
    Grad[:,i] = ((g_hat(np.multiply(beta,(1 + h*pos[:,i])), r) - g_hat(beta, r))/(beta[i]*h))[:,i]

In [8]:
cov = (1/T)*np.linalg.inv(Grad.transpose() @ np.linalg.inv(Omega) @ Grad)
stderr = np.sqrt(np.diag(cov))
testt = beta/stderr
p_value = 2*(1-norm.cdf(testt))

Test for the overidentified restrictions (Sargan/Hansen)

In [9]:
k = Omega.shape[0] # number of conditions on moment
r = cov.shape[0]   # number of parameters to be estimated
df = k-r
if k > r:
    stat_test = T*Init_func
    p_test = 1-chi2.cdf(stat_test,df)
else:
    stat_test = [ ]
    p_test = [ ]